In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import tree, preprocessing, svm, ensemble

In [2]:
train = pd.read_csv("data/train.csv", delimiter=' ', header=None)
labels = pd.read_csv("data/labes.csv", delimiter=',')
test = pd.read_csv("data/test.csv", delimiter=' ', header=None)

In [7]:
print(train.shape)
train.head(2)

(500, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201


# Exploratory analysis and preprocessing

### Categorical attributes

Frequencies of values among categorical attributes:

In [68]:
frequencies = {i + 1 : train[i].value_counts(sort=False).to_dict() for i in range(18) if train[i].dtype == 'O'}

In [70]:
for f in frequencies.items():
    print(str(f[0]) + " : " + str({k: "{0:.2f}".format(v / 500)  for (k, v) in f[1].items()}))

1 : {'A12': '0.26', 'A13': '0.07', 'A14': '0.40', 'A11': '0.27'}
3 : {'A32': '0.53', 'A34': '0.30', 'A31': '0.04', 'A33': '0.09', 'A30': '0.04'}
4 : {'A45': '0.02', 'A49': '0.11', 'A410': '0.01', 'A40': '0.23', 'A48': '0.01', 'A41': '0.11', 'A44': '0.01', 'A46': '0.04', 'A43': '0.28', 'A42': '0.18'}
17 : {'A173': '0.64', 'A174': '0.13', 'A172': '0.21', 'A171': '0.02'}
6 : {'A65': '0.17', 'A62': '0.10', 'A61': '0.61', 'A64': '0.05', 'A63': '0.07'}
7 : {'A72': '0.16', 'A73': '0.36', 'A75': '0.25', 'A74': '0.18', 'A71': '0.06'}
9 : {'A91': '0.05', 'A94': '0.09', 'A93': '0.53', 'A92': '0.33'}
10 : {'A101': '0.91', 'A102': '0.04', 'A103': '0.05'}
12 : {'A121': '0.30', 'A123': '0.33', 'A122': '0.23', 'A124': '0.14'}
14 : {'A143': '0.82', 'A142': '0.05', 'A141': '0.13'}
15 : {'A152': '0.71', 'A153': '0.09', 'A151': '0.20'}


Delete unimportant columns

In [67]:
train = train.drop([18, 19], axis=1)
train = test.drop([18, 19], axis=1)

Merge where possible

In [71]:
replacement = {'A13': 'A12', # has an account
               # 3 - Credit history
               'A31': 'A30', # no problems 
               'A34': 'A33', # some problems
               # 4 - Purpose
               'A45': 'A42', 'A44': 'A42', 'A43': 'A42', # home related
               'A410' : 'A43', 'A49': 'A43', 'A48': 'A43', 'A46': 'A43', # other
               # 6 - Savings
               'A64': 'A62', 'A63': 'A62', # > 100 DM
               # 7 - Present employment since
               'A72': 'A71', # < 1 year
               # 9 - marital status
               'A93': 'A91',
               # 14 - other installment plans
               'A142': 'A141',
               # 17 - job
               'A172': 'A171'}

In [72]:
train = train.replace(replacement)
test = test.replace(replacement)

In [88]:
len(np.unique(train.select_dtypes(include=['object']).values))

35

### Numerical attributes

In [105]:
# train.select_dtypes(exclude=['object']).hist()
# plt.show()

In [103]:
# Attribute 2 - credit duration
def f(x):
    return 0 if x <= 10 else 1 if x <= 20 else 2 if x <= 30 else 3
train[1] = train[1].apply(f)
# Attrbute 5 - credit amount
def f(x):
    return 0 if x <= 2000 else 1 if x <= 4000 else 2
train[4] = train[4].apply(f)
# Attribute 13 - age (years)
def f(x):
    return 0 if x <= 25 else 1 if x <= 35 else 2 if x <= 50 else 3
train[12] = train[12].apply(f)
# Attribute 16 - # of existing credits
def f(x):
    return 0 if x <= 1 else 1
train[15] = train[15].apply(f)

# Encode non-numerical labels

In [71]:
encoding = {'A' + str(i) : i % 10 for i in range(210)}
encoding['A410'] = 11

In [72]:
train = train.replace(encoding)
test = test.replace(encoding)

# Random Forest

In [50]:
rf_clf = ensemble.RandomForestClassifier()
rf_clf = rf_clf.fit(train, labels['Prediction'])

/usr/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/lib/p

In [51]:
test_labels = labels
test_labels['Prediction'] = rf_clf.predict_proba(test)

Something weird is happening

In [55]:
test_labels['Prediction'] = 1 - test_labels['Prediction']

In [56]:
test_labels.to_csv('manual_encoding_rf.csv', index=False)

# SVM

In [62]:
svm_clf = svm.SVC(probability=True)
svm_clf = svm_clf.fit(train, labels['Prediction'])

In [63]:
test_labels = labels
test_labels['Prediction'] = svm_clf.predict_proba(test)

In [64]:
test_labels.to_csv('manual_encoding_svc.csv', index=False)